# Collect DFT data into *, *O, *OH collections
---

Notes:
  * If there exists only a single slab for a particular adsorbate, and that slab has a averaged absolute magmom per atom of less than XXX, then we should check if there are slabs of different adsorbates in that set to tranplant the magmoms from

# Import Modules

In [1]:
import os
print(os.getcwd())
import sys

import pickle
import shutil
from pathlib import Path

import pandas as pd
import numpy as np

# #########################################################
from IPython.display import display

# #########################################################
from methods import get_df_jobs_anal
from methods import get_df_jobs_data
from methods import get_df_atoms_sorted_ind
from methods import get_df_magmoms
from methods import get_df_jobs_paths
from methods import get_df_jobs_oh_anal

# #########################################################
from local_methods import read_magmom_comp_data, save_magmom_comp_data
from local_methods import process_group_magmom_comp
from local_methods import get_oer_set
from local_methods import analyze_O_in_set

/home/raulf2012/Dropbox/01_norskov/00_git_repos/PROJ_IrOx_OER/dft_workflow/job_analysis/compare_magmoms


# Script Inputs

In [2]:
verbose = False
# verbose = True

redo_all_jobs = False
# redo_all_jobs = True

# Read Data

In [3]:
# #########################################################
df_jobs_anal = get_df_jobs_anal()

# #########################################################
df_jobs_data = get_df_jobs_data()

# #########################################################
df_atoms_sorted_ind = get_df_atoms_sorted_ind()

# #########################################################
magmom_data_dict = read_magmom_comp_data()

# #########################################################
df_magmoms = get_df_magmoms()

# #########################################################
df_jobs_paths = get_df_jobs_paths()

# #########################################################
df_magmoms = df_magmoms.set_index("job_id")

# #########################################################
df_jobs_oh_anal = get_df_jobs_oh_anal()

In [4]:
# df_jobs_oh_anal

# Preprocessing data objects

## Processing `df_jobs_anal` (only completed job sets, filter out *O)

In [5]:
# #########################################################
# Only completed jobs will be considered
df_jobs_anal_i = df_jobs_anal[df_jobs_anal.job_completely_done == True]

# #########################################################
# Remove the *O slabs for now
# The fact that they have NaN active sites will mess up the groupby
ads_list = df_jobs_anal_i.index.get_level_values("ads").tolist()
ads_list_no_o = [i for i in list(set(ads_list)) if i != "o"]

idx = pd.IndexSlice
df_jobs_anal_no_o = df_jobs_anal_i.loc[idx[:, :, ads_list_no_o, :, :], :]

# #########################################################
# Only keep OER job sets that have all adsorbates present and completed
indices_to_keep = []
groupby_cols = ["compenv", "slab_id", "active_site", ]
grouped = df_jobs_anal_no_o.groupby(groupby_cols)
for name_i, group in grouped:

    # print("TEMP")
    # index_i = ('slac', 'fagumoha_68', 'oh', 62.0, 3)
    # if index_i in group.index:
    #     print(name_i)

    group_index = group.index.to_frame()
    ads_list = list(group_index.ads.unique())
    oh_present = "oh" in ads_list
    bare_present = "bare" in ads_list
    all_req_ads_present = oh_present and bare_present
    if all_req_ads_present:
        indices_to_keep.extend(group.index.tolist())

df_jobs_anal_no_o_all_ads_pres = df_jobs_anal_no_o.loc[
    indices_to_keep    
    ]
df_i = df_jobs_anal_no_o_all_ads_pres

## Process `df_jobs_oh_anal`

In [6]:
df_jobs_oh_anal = df_jobs_oh_anal.set_index(
    ["compenv", "slab_id", "active_site", ], drop=False)

# Checking if there are OER sets that have slabs with magmom 0'ed out

In [7]:
# Cutoff for how low the magmoms of slab can go before I rerun with different spin
magmom_cutoff = 0.1

In [8]:
# #########################################################
verbose_local = False
# #########################################################

# #########################################################
data_dict_list = []
# #########################################################
groupby_cols = ["compenv", "slab_id", "active_site", ]
grouped = df_i.groupby(groupby_cols)
for i_cnt, (name_i, group) in enumerate(grouped):
    data_dict_i = dict()

    if verbose_local:
        print(40 * "*")
        print("name_i:", name_i)

    # #########################################################
    compenv_i = name_i[0]
    slab_id_i = name_i[1]
    active_site_i = name_i[2]
    # #########################################################


    # #########################################################
    group_i = get_oer_set(
        group=group,
        compenv=compenv_i,
        slab_id=slab_id_i,
        df_jobs_anal=df_jobs_anal,
        )

    # #########################################################
    magmom_data_out = analyze_O_in_set(
        data_dict_i,
        group_i,
        df_magmoms,
        magmom_cutoff=magmom_cutoff,
        compenv=compenv_i,
        slab_id=slab_id_i,
        active_site=active_site_i,
        )

    # #####################################################
    data_dict_i.update(magmom_data_out)
    # #####################################################
    data_dict_list.append(data_dict_i)
    # #####################################################

# #########################################################
df_m = pd.DataFrame(data_dict_list)
df_m = df_m.set_index(["compenv", "slab_id", "active_site", ], drop=False)

In [9]:
# data_dict_i.update(
# data_dict_tmp
# )

In [10]:
# data_dict_i

In [11]:
df_m.head()

compenv      slab_id  active_site  \
compenv  slab_id     active_site                                       
sherlock fogalonu_46 16.0         sherlock  fogalonu_46         16.0   
         kenukami_73 84.0         sherlock  kenukami_73         84.0   
                     86.0         sherlock  kenukami_73         86.0   
         kipatalo_90 78.0         sherlock  kipatalo_90         78.0   
                     81.0         sherlock  kipatalo_90         81.0   

                                  *O_w_low_magmoms  *O_w_not_low_magmoms  
compenv  slab_id     active_site                                          
sherlock fogalonu_46 16.0                     True                 False  
         kenukami_73 84.0                     True                 False  
                     86.0                     True                 False  
         kipatalo_90 78.0                     True                 False  
                     81.0                     True                 False

In [12]:
# assert False

In [13]:
index_diff_0 = df_jobs_oh_anal.index.difference(df_m.index)
index_diff_1 = df_m.index.difference(df_jobs_oh_anal.index)

# index_diff_0.shape[0]

mess_i = "This shouldn't be, look into it"
assert index_diff_1.shape[0] == 0, mess_i

# #########################################################
shared_index = df_jobs_oh_anal.index.intersection(df_m.index)

df_jobs_oh_anal = df_jobs_oh_anal.loc[shared_index]
df_m = df_m.loc[shared_index]

In [14]:
# df_m.index.difference?

# df_jobs_oh_anal

In [15]:
# index_diff_0
# index_diff_1

In [16]:
# assert False

In [17]:
list_0 = list(df_m.columns)
list_1 = list(df_jobs_oh_anal.columns)

shared_cols = list(set(list_0).intersection(list_1))

# shared_cols
# df_m.drop(columns=shared_cols)

df_list = [
    df_m.drop(columns=shared_cols),
    df_jobs_oh_anal,
    ]

df_m2 = pd.concat(df_list, axis=1)
df_m2 = df_m2.sort_index()

In [18]:
df_m2.head()

*O_w_low_magmoms  *O_w_not_low_magmoms  \
compenv  slab_id     active_site                                           
sherlock fogalonu_46 16.0                     True                 False   
         kenukami_73 84.0                     True                 False   
                     86.0                     True                 False   
         kipatalo_90 78.0                     True                 False   
                     81.0                     True                 False   

                                   compenv      slab_id  active_site  \
compenv  slab_id     active_site                                       
sherlock fogalonu_46 16.0         sherlock  fogalonu_46         16.0   
         kenukami_73 84.0         sherlock  kenukami_73         84.0   
                     86.0         sherlock  kenukami_73         86.0   
         kipatalo_90 78.0         sherlock  kipatalo_90         78.0   
                     81.0         sherlock  kipatalo_90         81.0   

                                  all_oh_attempts_done  \
compenv  slab_id     active_site                         
sherlock fogalonu_46 16.0                        False   
         kenukami_73 84.0                         True   
                     86.0                         True   
         kipatalo_90 78.0                        False   
                     81.0                         True   

                                                              job_ids_sorted_energy  \
compenv  slab_id     active_site                                                      
sherlock fogalonu_46 16.0         [rataruha_18, lewudaha_47, marewuhi_22, tikili...   
         kenukami_73 84.0         [gawopigo_86, homimesi_72, dinagiso_25, bapere...   
                     86.0         [pagugovi_98, lafapulu_00, gotigase_31, benisi...   
         kipatalo_90 78.0                   [masoroko_92, mugorodi_48, lirusemu_36]   
                     81.0         [rahulesi_46, neriruwi_42, hekihonu_06, vapira...   

                                 job_id_most_stable  
compenv  slab_id     active_site                     
sherlock fogalonu_46 16.0               rataruha_18  
         kenukami_73 84.0               gawopigo_86  
                     86.0               pagugovi_98  
         kipatalo_90 78.0               masoroko_92  
                     81.0               rahulesi_46

In [19]:
df_m3 = df_m2[
    (df_m2["*O_w_low_magmoms"] == True) & \
    (df_m2["*O_w_not_low_magmoms"] == False) & \
    (df_m2["all_oh_attempts_done"] == True) & \
    [True for i in range(len(df_m2))]
    ]

data_dict_list = []
for i_cnt, row_i in df_m3.iterrows():
    data_dict_i = dict()
    
    # #####################################################
    compenv_i = row_i.compenv
    slab_id_i = row_i.slab_id
    active_site_i = row_i.active_site
    all_oh_attempts_done_i = row_i.all_oh_attempts_done
    job_ids_sorted_energy_i = row_i.job_ids_sorted_energy
    job_id_most_stable_i = row_i.job_id_most_stable
    # #####################################################

    # #####################################################
    row_magmoms_i = df_magmoms.loc[job_id_most_stable_i]
    # #####################################################
    sum_abs_magmoms_pa_i = row_magmoms_i.sum_abs_magmoms_pa
    # #####################################################

    # print("sum_abs_magmoms_pa_i:", sum_abs_magmoms_pa_i)

    rerun_from_oh = False
    # if sum_abs_magmoms_pa_i > magmom_cutoff:
    if sum_abs_magmoms_pa_i > 0.07:
        rerun_from_oh = True

    # #####################################################
    data_dict_i["compenv"] = compenv_i
    data_dict_i["slab_id"] = slab_id_i
    data_dict_i["active_site"] = active_site_i
    data_dict_i["rerun_from_oh"] = rerun_from_oh
    # data_dict_i["all_oh_attempts_done_i"] = all_oh_attempts_done_i
    # data_dict_i["job_ids_sorted_energy_i"] = job_ids_sorted_energy_i
    # data_dict_i["job_id_most_stable_i"] = job_id_most_stable_i
    # data_dict_i[""] = 
    # #####################################################
    data_dict_i.update(row_i.to_dict())
    # #####################################################
    data_dict_list.append(data_dict_i)
    # #####################################################

df_rerun_from_oh = pd.DataFrame(data_dict_list)

In [20]:
df_rerun_from_oh.head()
# df_rerun_from_oh

,compenv,slab_id,active_site,rerun_from_oh,*O_w_low_magmoms,*O_w_not_low_magmoms,all_oh_attempts_done,job_ids_sorted_energy,job_id_most_stable
0,sherlock,kenukami_73,84.0,True,True,False,True,"[gawopigo_86, homimesi_72, dinagiso_25, bapere...",gawopigo_86
1,sherlock,kenukami_73,86.0,True,True,False,True,"[pagugovi_98, lafapulu_00, gotigase_31, benisi...",pagugovi_98
2,sherlock,kipatalo_90,81.0,True,True,False,True,"[rahulesi_46, neriruwi_42, hekihonu_06, vapira...",rahulesi_46
3,sherlock,tofiwadi_49,45.0,True,True,False,True,"[furugahi_30, giborehu_66, popapuhu_54, kikiwo...",furugahi_30
4,sherlock,vuvunira_55,65.0,True,True,False,True,"[dusanala_91, kokamume_46, ribavane_13, nafovi...",dusanala_91


# Save data to pickle

In [21]:
# Pickling data ###########################################
import os; import pickle
directory = os.path.join(
    os.environ["PROJ_irox_oer"],
    "dft_workflow/job_analysis/compare_magmoms",
    "out_data")
if not os.path.exists(directory): os.makedirs(directory)
path_i = os.path.join(directory, "df_rerun_from_oh.pickle")
with open(path_i, "wb") as fle:
    pickle.dump(df_rerun_from_oh, fle)
# #########################################################

In [22]:
from methods import get_df_rerun_from_oh
df_rerun_from_oh_tmp = get_df_rerun_from_oh()
df_rerun_from_oh_tmp.head()

,compenv,slab_id,active_site,rerun_from_oh,*O_w_low_magmoms,*O_w_not_low_magmoms,all_oh_attempts_done,job_ids_sorted_energy,job_id_most_stable
0,sherlock,kenukami_73,84.0,True,True,False,True,"[gawopigo_86, homimesi_72, dinagiso_25, bapere...",gawopigo_86
1,sherlock,kenukami_73,86.0,True,True,False,True,"[pagugovi_98, lafapulu_00, gotigase_31, benisi...",pagugovi_98
2,sherlock,kipatalo_90,81.0,True,True,False,True,"[rahulesi_46, neriruwi_42, hekihonu_06, vapira...",rahulesi_46
3,sherlock,tofiwadi_49,45.0,True,True,False,True,"[furugahi_30, giborehu_66, popapuhu_54, kikiwo...",furugahi_30
4,sherlock,vuvunira_55,65.0,True,True,False,True,"[dusanala_91, kokamume_46, ribavane_13, nafovi...",dusanala_91


In [33]:
# assert False

# Writing the slabs with the smallest magmoms to file to manually inspect

In [32]:
df_i = df_magmoms[df_magmoms.sum_abs_magmoms_pa > 1e-5]
df_i = df_i.sort_values("sum_abs_magmoms_pa", ascending=True)

for i_cnt, (job_id_i, row_i) in enumerate(df_i.iloc[0:20].iterrows()):

    # #####################################################
    row_paths_i = df_jobs_paths.loc[job_id_i]
    # #####################################################
    gdrive_path_i = row_paths_i.gdrive_path
    # #####################################################

    path_i = os.path.join(
        os.environ["PROJ_irox_oer_gdrive"],
        gdrive_path_i,
        "final_with_calculator.traj")

    directory = os.path.join(
        "__temp__/low_magmom_slabs")

    if not os.path.exists(directory):
        os.makedirs(directory)

    out_path = os.path.join(
        directory,
        str(i_cnt).zfill(3) + "_" + job_id_i + ".traj")

    shutil.copyfile(
        path_i,
        out_path)

# df_i.iloc[0:20]

In [ ]:
# #########################################################
print(20 * "# # ")
print("All done!")
print("analyse_jobs.ipynb")
print(20 * "# # ")
# assert False
# #########################################################